# Session 4: Building Dashboards with Streamlit

[Streamlit](streamlit.io) is a tool for creating webapps and dashboards in pure python.

## Getting Started

Let's start with a hello-world.

In [37]:
%%file hello.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

st.title("Hello, Streamlit!")

st.header("The Dataset")

df = pd.read_csv("data/un-min.csv")

st.write(df.head(10))

# st.write(df[df.region == "Asia"])

st.header("And everything else")

st.write("Hello, world")
st.write(["One", "Two", "Three"])
st.write({"x": 1, "y": 2})




Overwriting hello.py


Open your terminal and run:

```
streamlit run hello.py
```

### Charts

Streamlit support its own interactive charts and matplotlib.

In [38]:
%%file charts.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

st.title("Charts")

st.header("The Dataset")

df = pd.read_csv("data/un-min.csv")

st.write(df.head())

st.header("Charts")

st.subheader("Streamlit")    
st.scatter_chart(df, x="GDPperCapita", y="infantMortality")

st.subheader("Matplotlib")        
ax = df.plot.scatter(x="GDPperCapita", y="infantMortality")
st.pyplot(ax.figure)

st.subheader("Seaborn")
ax = sns.scatterplot(df, x="GDPperCapita", y="infantMortality", hue="region")
st.pyplot(ax.figure)

Overwriting charts.py


Run this application using:

```
$ streamlit run charts.py
```

### Input Widgets

Streamlit provides many input widgets which are easy to use.

In [49]:
%%file inputs.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

st.title("Input Widgets")

df = pd.read_csv("data/un-min.csv")

x = st.selectbox("X", list(df.columns))
y = st.selectbox("Y", list(df.columns))

color = st.selectbox("Color By", list(df.columns))

show = st.button("Show")

st.write("X =", x)
st.write("Y =", y)
st.write("Color =", color)
st.write("show =", show)

if show:
    ax = sns.scatterplot(df, x=x, y=y, hue=color)
    st.pyplot(ax.figure)


num = st.number_input("Number")
radio = st.radio("Direction", ["East", "West", "North", "South"])

st.write("Num", num)
st.write("Radio", radio)

Overwriting inputs.py


## Sidebar & Columns

In [48]:
%%file sidebar.py

import streamlit as st
import pandas as pd
import seaborn as sns

st.title("Sidebar & Columns")

df = pd.read_csv("data/un-min.csv")


with st.sidebar:
    st.header("Sidebar")

    x = st.selectbox("X", list(df.columns))
    y = st.selectbox("Y", list(df.columns))
    color = st.selectbox("Color By", list(df.columns))

    st.write("X =", x)
    st.write("Y =", y)
    st.write("Color =", color)
    # st.write("show =", show)

c1, c2 = st.columns(2)
with c1:
    st.subheader("Column 1")
    st.write([1, 2, 3, 4, 5])

with c2:
    st.subheader("Column 2")

    ax = sns.scatterplot(df, x=x, y=y, hue=color)
    st.pyplot(ax.figure)

Overwriting sidebar.py


#### Problem: Calculator

Write a streamlit application, that asks the user for two numbers x and y, select an operation from Add or Multiply and show the result.


Hint:

* [st.number_input][1]
* [st.radio][2]

[1]: https://docs.streamlit.io/library/api-reference/widgets/st.number_input
[2]: https://docs.streamlit.io/library/api-reference/widgets/st.radio

## Data Explorer v1

In [96]:
%%file data-explorer-v1.py
import streamlit as st
import pandas as pd
import seaborn as sns
from pathlib import Path

filters = {}

def select_dataset():
    st.header("Select Dataset")

    datasets = Path("data").glob("*.csv")
    datasets = list(datasets)

    p = st.selectbox("Select Dataset", datasets)
    return p


def select_columns(df):
    st.header("Select Columns")
    numeric_columns = df.select_dtypes(include="number").columns.tolist()
    x = st.selectbox("X", numeric_columns)
    y = st.selectbox("Y", numeric_columns)
    color = st.selectbox("Color By", list(df.columns))
    return x, y, color

def filter_numeric_column(df, name):
    c = df[name]
    left, right = st.slider(
        name, 
        min_value=c.min(), 
        max_value=c.max(),
        value=[c.min(), c.max()])

    return df[(c >= left) & (c <= right)]

def filter_categorical_column(df, name):
    st.subheader(name)
    c = df[name]

    cats = {cat: st.checkbox(str(cat)) for cat in c.unique()}
    selected = [k for k in cats if cats[k]]

    if not selected:
        return df

    return df[c.isin(selected)]


def find_categorical_columns(df):
    return [name for name in df.select_dtypes(exclude="number") 
            if df[name].nunique() <= 20]

def filter_dataset(df):
    st.header("Filters")

    for name in df.select_dtypes(include="number"):
        df = filter_numeric_column(df, name)

    cat_columns = find_categorical_columns(df)

    for name in cat_columns:
        df = filter_categorical_column(df, name)

    return df
    
with st.sidebar:
    st.title("Data Explorer")
    path = select_dataset()
    df = pd.read_csv(path)
    x, y, color = select_columns(df)

    df = filter_dataset(df)
    
title = path.stem.title()
st.title(title)

st.write(df.head())

st.header("Charts")
ax = sns.scatterplot(df, x=x, y=y, hue=color)
st.pyplot(ax.figure)

Overwriting data-explorer-v1.py


**Problem:** Add a Color By Column to the data explorer.

To get all categotical columns, you can use:

```
df.select_dtypes(exclude="numeric").columns.tolist()
```

**Problem:** Find all non-numeric columns that have less than 20 catagories and show checkboxes for each category to select that category.

If you select the un-min dataset, it would show checkboxes for each region and let's you turn on or off a region.